In [13]:
import pandas as pd
import numpy as np

import collections
import torch
import pickle
import time
import json
import torch.nn as nn
import torch.nn.functional as F
from  gensim.models import KeyedVectors
from featurization import word2vec, featurizer
import model as m
from featurizer import ChungusSet 
from torch.utils.data import DataLoader
import geopy
from geopy.distance import geodesic
from torch.utils.data import Dataset

In [2]:
# load word embeddings
def load_embedding():
    # load word2vec weights
    EpochSaver = word2vec.EpochSaver
    w2v = KeyedVectors.load_word2vec_format('./models/word2vec.model')

    weights = torch.FloatTensor(w2v.vectors)
    embedding = nn.Embedding.from_pretrained(weights)

    return embedding

embedding = load_embedding()

INFO - 01:45:32: loading projection weights from ./models/word2vec.model
INFO - 01:46:10: loaded (126352, 300) matrix from ./models/word2vec.model


In [20]:
# load model
path = "epoch_74.model"
model = m.Classifier(embedding)
model = nn.DataParallel(model, device_ids=[0,1,2,3,4,5,6,7])
checkpoint = torch.load(path)
model.load_state_dict(checkpoint['state_dict'])
model = model.to("cuda:0")
batch_size = 256

In [17]:
def evaluate_model(model):
    test_set = pickle.load(open("data/test_set.p", "rb"))
    test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True,num_workers=30, drop_last=True)
    print("Test set of %d entries" % len(test_set))
    err=0
    tot = 0
    with torch.no_grad():
        i = 0
        for (words, subs, times, labels) in test_loader:
            # remove subs, see how model does
            # subs = torch.zeros(batch_size, 5000)   
            
            output = model(words, subs, times, labels)
        
            # let the maximum index be our predicted class
            _, yh = torch.max(output, 1)

            tot += labels.size(0)
            
            i+=1
            if(i % 10 == 0):
                print("Iteration: %d, processed %d labels" % (i, tot))
                print(labels.size())
            
            ## add to err number of missclassification, i.e. number of indices that
            ## yh and y are not equal
            ## note that y and yh are vectors of size = batch_size = (256 in our case)
            err += sum(list(map(lambda i: 1 if labels[i] != yh[i] else 0, range(len(labels)))))

    print('Accuracy of FC prediction on test users: %5.2f%%' % (100-100 * err / tot))

In [ ]:
evaluate_model(model)

Test set of 10755 entries


In [12]:
# 55: 57.99
# 58: 58.18
# 60: 58.28
# 69: 58.80%
# 70: 59.54
# 73: 58.71

print(len(test_set))

NameError: name 'test_set' is not defined